# Using `libfec` to (almost) replicate an LA Times campaign finance story

Gabrielle LaMarr LeMee and Sandhya Kambhampati of the LA Times published 
[*"More than half of Harris donors had not given to Biden’s 2024 campaign"*](https://www.latimes.com/politics/story/2024-08-27/where-kamala-harris-most-new-donors-july-2024-election)
on August 27th of 2024. It details the composition of Kamala Harris supporters in the weeks following Biden's resignation
and subsequent endorsement of Kamala Harris' presidential campaign. 

Let's try to recreate the story, using the `libfec` CLI and SQLite!

## Small caveat: not a full replication

The LA Times article uses a few different committee filings to source their data, while here we will only focus on receipts reported 
by ActBlue. This is nearly everything, but the data found here will be a slight undercount compared to the LA Times analysis. 

## Step 1: Sourcing all ActBlue data

The LAT article contains this nerdbox about the dataset they analyzed:

> *The Times analysis is based on Federal Election Commission filings as of Aug. 20. The Times analysis includes itemized contributions to **Harris for President, Harris Victory Fund, Harris Baldwin Victory Fund, and Harris Action Fund**. Some contributions to these committees were **given through ActBlue**. A donor was determined to be a prior Biden donor if a donation from their unique combination of first name, last name and ZIP Code had been made **from Jan. 2023 to July 20, 2024**.*

So we're looking at data as early as January 2023, from ActBlue (for this replication), and for those four specific committees. 

We can use `libfec` to download this data directly from the ActBlue filings, we'll just need to find the relevent FEC filing IDs. Looking at [the ActBlue FEC commitee webpage](https://docquery.fec.gov/cgi-bin/forms/C00401224/), we can see all the filings that ActBlue have submitted, and we care about the `F3X` forms. This is [FEC Form 3X](https://www.fec.gov/resources/cms-content/documents/policy-guidance/fecfrm3x.pdf), aka "Report of Receipts and Disbursements", that committees like ActBlue file. 

Here we care about the following rows from that table:


- `FEC-1812188`: AUG MONTHLY 	(`07/01/2024`to `07/31/2024`)
- `FEC-1805179`: JUL MONTHLY 	(`06/01/2024`to `06/30/2024`)
- `FEC-1791562`: JUN MONTHLY 	(`05/01/2024`to `05/31/2024`)
- `FEC-1785179`: MAY MONTHLY 	(`04/01/2024`to `04/30/2024`)
- `FEC-1779040`: APR MONTHLY 	(`03/01/2024`to `03/31/2024`)
- `FEC-1765652`: MAR MONTHLY 	(`02/01/2024`to `02/29/2024`)
- `FEC-1758569`: FEB MONTHLY 	(`01/01/2024`to `01/31/2024`)
- `FEC-1752852`: YEAR-END 	(`07/01/2023`to `12/31/2023`)
- `FEC-1720554`: MID-YEAR 	(`01/01/2023`to `06/30/2023`)

These cover the entire time range (2023 to the present) that we care about.

To access the itemizations from these filings, we can use the `libfec` CLI to download and parse these filings without needing to download them manually. 
We could provide these IDs one-by-one on the command line, or we can add them to an "input file" that `libfec` can read directly. 
We do that here, placing these IDs in a text file called `actblue-202301-202407.txt`:

In [ ]:
! cat actblue-202301-202407.txt

Now let's download these filings with the `libfec` CLI:


```bash
libfec export -i actblue-202301-202407.txt --target schedule-a -o actblue-202301-202407.db
```

A bit more about what this command does:
- The `export` command will export all the itemizations in the provided FEC filings to a SQLite database
- The `-i actblue-202301-202407.txt` argument tells `libfec` to download the filing IDs present in the `actblue-202301-202407.txt` file, which we prepared above. 
- The `--target schedule-a` argument tells `libfec` to only export Schedule A itemizations, which are "receipts" (contributions). A single `libfec_schedule_a` table will be created. 
- The `-o actblue-all.db` argument is the path to the output SQLite database, that `libfec` will create and populate. 

This will take some time! It's `~28GB` of data to download, and the output SQLite database will be `~15 GB` large.

Once complete, we will have a large `actblue-202301-202407.db` SQLite database with a single large table: `libfec_schedule_a`. 

For the rest of this notebook, we're done using `libfec`! We have all the data we need in this SQLite database. 
We can filter and aggregate this dataset as needed using pure SQL. 
You could also use other data analysis tools in other programming languages (Python+Pandas, R+dpylr, JavaScript+Observable), all of which support querying SQLite databases.

In this notebook, We'll use [JupySQL](https://jupysql.ploomber.io/en/latest/quick-start.html) to keep everything in pure SQL,
to make it easier to copy+paste into your favorite tools. 

## Step 2: Filter out to just Biden/Harris 2023-2024 contributions

ActBlue filings contain a lot of data than just Biden/Harris, so we'll need to filter down to just the donations we care about. 
We can still keep the large `actblue-202301-202407.db` as-is, we'll create a new SQLite database called `biden-harris-202301-202407.db` 
and copy over the data we need. 

In [2]:
%load_ext sql
%sql sqlite:///biden-harris-202301-202407.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Let's copy over the data from the the big ActBlue database to this one. 
Along the way, we will filter for only Biden/Harris contributions, and add
a few extra columns to help with analysis later on. 

In [3]:
%%sql
attach database 'actblue-202301-202407.db' as source;

create table biden_harris_itemizations as
  select
    -- The original contributor_zip_code column can sometimes contain +4 digits,
    -- and for our purposes, we only care about the first 5 digits (ex 90210)
    substr(contributor_zip_code, 1, 5) as contributor_zip_code5,

    -- We don't have any unique identifiers for individual contributors. The LA Times
    -- analysis estimated one by using a "unique combination of first name, last name,
    --  and ZIP Code".
    lower(
      format(
        '%s%s%s',
        trim(contributor_first_name),
        trim(contributor_last_name),
        substr(contributor_zip_code, 1, 5)
      )
    ) as contributor_id,

    -- copy over the rest of the columns in libfec_schedule_a
    *
  from source.libfec_schedule_a
  where
    -- We only care about donations made to Biden/Harris campaigns, and not
    -- all ActBlue donations. We can filter them out by looking for Harris
    -- committee IDs as they appear in the "memo_text_description" column,
    -- which contains values like 'Earmarked for BIDEN FOR PRESIDENT (C00703975)'.
    -- These 4 committees were specifically mentioned in the LA Times article nerdbox.
    (
      -- "HARRIS FOR PRESIDENT"
      memo_text_description like '%C00703975%'
      -- "HARRIS VICTORY FUND"
      or memo_text_description like '%C00744946%'
      -- "HARRIS BALDWIN VICTORY FUND"
      or memo_text_description like '%C00849281%'
      -- "HARRIS ACTION FUND"
      or memo_text_description like '%C00838912%'
  );


Running query in 'sqlite:///biden-harris-202301-202407.db'

++
||
++
++

Now the `biden_harris_itemizations` table contains all contributions to Biden/Harris campaigns since 2023! 
Here's a sample of what that table looks like, with the additional `contributor_zip_code5` and `contributor_id` 
columns we added. 

In [4]:
%%sql
select * from biden_harris_itemizations limit 10;

Running query in 'sqlite:///biden-harris-202301-202407.db'

contributor_zip_code5,contributor_id,filing_id,form_type,filer_committee_id_number,transaction_id,back_reference_tran_id_number,back_reference_sched_name,entity_type,contributor_organization_name,contributor_last_name,contributor_first_name,contributor_middle_name,contributor_prefix,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip_code,election_code,election_other_description,contribution_date,contribution_amount,contribution_aggregate,contribution_purpose_descrip,contributor_employer,contributor_occupation,donor_committee_fec_id,donor_committee_name,donor_candidate_fec_id,donor_candidate_last_name,donor_candidate_first_name,donor_candidate_middle_name,donor_candidate_prefix,donor_candidate_suffix,donor_candidate_office,donor_candidate_state,donor_candidate_district,conduit_name,conduit_street1,conduit_street2,conduit_city,conduit_state,conduit_zip_code,memo_code,memo_text_description,reference_code
30127,da babya30127,1720554,SA11AI,C00401224,SA11AI_535329455,,,IND,,A,DA BABY,,,,122 DEERHUNTER LN,,POWDER SPRINGS,GA,30127,,,2023-06-30,1.0,1.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),
92011,dianaa92011,1720554,SA11AI,C00401224,SA11AI_534850611,,,IND,,A,DIANA,,,,6808 ZINNIA CT,,CARLSBAD,CA,92011,,,2023-06-28,10.0,10.0,Earmark,ANTHEM,PROJECT DIRECTOR,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),
95008,johna95008,1720554,SA11AI,C00401224,SA11AI_529522643,,,IND,,A,JOHN,,,,1636 ADRIEN DR,,CAMPBELL,CA,95008,,,2023-05-19,10.0,10.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),
20016,ka20016,1720554,SA11AI,C00401224,SA11AI_526451881,,,IND,,A,K,,,,3713 YUMA ST NW,,WASHINGTON,DC,20016,,,2023-04-25,5.0,5.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN VICTORY FUND (C00744946),
06109,luisa06109,1720554,SA11AI,C00401224,SA11AI_531348463,,,IND,,A,LUIS,,,,106 STILLWOLD DRIVE,,WETHERSFIELD,CT,06109,,,2023-06-02,25.0,25.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),
32615,marya32615,1720554,SA11AI,C00401224,SA11AI_529907797,,,IND,,A,MARY,,,,15817 NW 173 RD STREET,,ALACHUA,FL,32615,,,2023-05-23,10.0,10.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN VICTORY FUND (C00744946),
10451,michaela10451,1720554,SA11AI,C00401224,SA11AI_530180169,,,IND,,A,MICHAEL,,,,940 GRAND CONCOURSE,,NEW YORK,NY,10451,,,2023-05-25,10.0,10.0,Earmark,EONE,PRODUCER,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),
50310,garya adams50310,1720554,SA11AI,C00401224,SA11AI_527825689,,,IND,,A ADAMS,GARY,,,,3411 48TH STREET,,DES MOINES,IA,503103219,,,2023-05-04,25.0,25.0,Earmark,ISL EDUCATION LENDING,SCHOOL SERVICES LIAISON,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),
98116,juliea alley98116,1720554,SA11AI,C00401224,SA11AI_531049324,,,IND,,A ALLEY,JULIE,,,,5208 SW GENESEE ST,,"SEATTLE, WA",WA,98116,,,2023-05-31,25.0,25.0,Earmark,LIFELONG,IT MANAGER,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),
21668,sharona andrews21668,1720554,SA11AI,C00401224,SA11AI_526440924,,,IND,,A ANDREWS,SHARON,,,,1304 FOXX CT,,SUDLERSVILLE,MD,21668,,,2023-04-25,10.0,20.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),


## Step 3: Compute contributor stats

The above table is 1 row per contribution, meaning the same individual appears multiple times if they contributed multiple times. 
In this analysis, we care about whether or not a contributor gave to both Biden or Harris, depending on when their contribution took place
(before or after `2024-07-21`, when Biden dropped out).

To aid in this, we will create a temporary table called `biden_harris_contributor_stats` that pre-computes these stats,
with 1 row per unique contributor. 

In [5]:
%%sql

-- A temporary table that pre-compute per-contributor statistics, to aid in later analysis.
-- 1 row per unique contributor, based on their first name/last name/zipcode.
create temp table biden_harris_contributor_stats as
  select
    contributor_id,
    contributor_state,
    contributor_zip_code5,

    -- if the contributor donated at least once to Biden
    count(*) filter (where contribution_date < '2024-07-21') > 0 as biden_donor,

    -- if the contributor donated at least once to Harris
    count(*) filter (where contribution_date >= '2024-07-21') > 0 as harris_donor,

    -- Total amount the contributor had given to Biden
    sum(contribution_amount) filter (where contribution_date < '2024-07-21')  as biden_contribution_amount,

    -- Total amount the contributor has given to Harris
    sum(contribution_amount) filter (where contribution_date >= '2024-07-21') as harris_contribution_amount,

    -- Total amount the contributor has given to either Biden or Harris
    sum(contribution_amount) as total_contribution_amount

  from biden_harris_itemizations
  group by 1;

Running query in 'sqlite:///biden-harris-202301-202407.db'

++
||
++
++

In [6]:
%%sql
select * from biden_harris_contributor_stats limit 10;

Running query in 'sqlite:///biden-harris-202301-202407.db'

contributor_id,contributor_state,contributor_zip_code5,biden_donor,harris_donor,biden_contribution_amount,harris_contribution_amount,total_contribution_amount
! mariaalfaro-brooks95492,CA,95492,0,1,None,500.0,500.0
!molliefair29412,SC,29412,1,0,100.0,None,100.0
# briansheehan80020,CO,80020,1,1,25.0,75.0,100.0
#2 douglas shimmerbrown23456,VA,23456,1,0,125.0,None,125.0
#balcurtis92651,CA,92651,1,0,200.0,None,200.0
#balwright17815,PA,17815,0,1,None,35.0,35.0
#minhome91401,CA,91401,1,0,381.0,None,381.0
'ameliakolokihakaufisi94555,CA,94555,0,1,None,50.0,50.0
'bunmiojugbele11580,NY,11580,0,1,None,50.0,50.0
'clyde 'chuck'fallaw iii29036,SC,29036,1,0,270.0,None,270.0


Now time for a sanity check: How many contributors have donated to Harris? 
With this new table, it's a single SQL statement:

In [7]:
%%sql
select sum(harris_donor) from biden_harris_contributor_stats;

Running query in 'sqlite:///biden-harris-202301-202407.db'

sum(harris_donor)
2183840


2.1 million unique donors! The LA Times analysis says 2.6 million, but remember, this is only ActBlue filings so likely an undercount. 

## Cross-checking LA Times numbers

Let's go over a few claims that the LA Times article makes and replicate them. The first one:

 > "...70% of her support was from donors who had not given previously to President Biden..."

 We can calculate that with:

In [ ]:
%%sql
select
  1.0 * sum(harris_donor and not biden_donor) / sum(harris_donor) as harris_not_biden_ratio
from biden_harris_contributor_stats;

They claimed 70%, we get 69.7%. Pretty close!

Now let's try to recreate this graphic from the article:

![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2010.42.47%20AM.png)

Now this is a state-level map where we calculate the ratio of Harris-only supports to Harris-and-Biden supporters. We can calculate that with:

In [12]:

%%sql data =<<
select
  contributor_state,
  1.0 * sum(harris_donor and not biden_donor) / sum(harris_donor) as harris_not_biden_ratio
from biden_harris_contributor_stats
group by 1
order by 1;

Running query in 'sqlite:///biden-harris-202301-202407.db'

contributor_state,harris_not_biden_ratio
AA,0.8070175438596491
AB,0.6176470588235294
AE,0.7888707037643208
AG,0.0
AK,0.6767988601282355
AL,0.7188507768982703
AP,0.7811158798283262
AR,0.717209099814956
AS,0.75
AV,1.0


In [13]:
data.csv("harris-ratio-by-state.csv")

CSV results at /Users/alex/projects/libfec/examples/latimes-harris-new/harris-ratio-by-state.csv

(Note that some of these two-letter state codes refer to other territories and countries other than the 50 states)

Now that query was exported to a CSV file, which I then uploaded to Observable and [created this map](https://observablehq.com/d/868c7fa3a8779665), which looks like:

|Original|Recreation
|-|-
| ![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2010.42.47%20AM.png) |![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2011.26.54%20AM.png)

Pretty good, and very close to the LA Times map!


Now let's try to re-create this graphic, looking at specifically July 2024 with daily counts of Biden/Harris/Biden&Harris supporters:

![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2010.44.45%20AM.png)

In [15]:

%%sql data =<<
with biden_donors as materialized (
  select contributor_id
  from biden_harris_contributor_stats
  where biden_donor
)

-- July 1st - 20th, Biden-only donors
select
  contribution_date,
  'biden_only' as type,
  count(distinct contributor_id) as count
from biden_harris_itemizations
where contribution_date between '2024-07-01' and '2024-07-20'
group by 1

union all

-- July 1st - 30th, Biden and Harris donors
select
  contribution_date,
  'biden_and_harris' as type,
  count(distinct contributor_id) filter (where contributor_id in biden_donors) as count
from biden_harris_itemizations
where contribution_date between '2024-07-21' and '2024-08-01'
group by 1

union all

-- July 1st - 30th, Harris-only donors
select
  contribution_date,
  'harris_only' as type,
  count(distinct contributor_id) filter (where contributor_id not in biden_donors) as count
from biden_harris_itemizations
where contribution_date between '2024-07-21' and '2024-08-01'
group by 1;

Running query in 'sqlite:///biden-harris-202301-202407.db'

contribution_date,type,count
2024-07-01,biden_only,48058
2024-07-02,biden_only,46386
2024-07-03,biden_only,40518
2024-07-04,biden_only,32092
2024-07-05,biden_only,34302
2024-07-06,biden_only,34812
2024-07-07,biden_only,22416
2024-07-08,biden_only,29382
2024-07-09,biden_only,25850
2024-07-10,biden_only,23328


In [16]:
data.csv("july-by-candidate.csv")

CSV results at /Users/alex/projects/libfec/examples/latimes-harris-new/july-by-candidate.csv

Uploaded that CSV to [this notebook](https://observablehq.com/d/3e377c8a1749d69e), and our result:

|Original|Recreation
|-|-
| ![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2010.44.45%20AM.png) | ![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2011.30.48%20AM.png)

Finally, let's try to recreate this chart:
![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2010.43.50%20AM.png)

In [27]:
%%sql data =<<
with daily as (
select
  contribution_date,
  iif(contribution_date < '2024-07-21', 'biden', 'harris') as target,
  'small' as contribution_type,
  sum(contribution_amount) as contribution_amount
from biden_harris_itemizations
where contribution_amount < 200
group by 1

union all

select
  contribution_date,
  iif(contribution_date < '2024-07-21', 'biden', 'harris') as target,
  'large' as contribution_type,
  sum(contribution_amount) as contribution_amount
from biden_harris_itemizations
where contribution_amount >= 200
group by 1
)
select
  *,
  target || contribution_type as color,
  sum(contribution_amount) over (partition by contribution_type order by contribution_date) as value
from daily
order by contribution_date;

Running query in 'sqlite:///biden-harris-202301-202407.db'

contribution_date,target,contribution_type,contribution_amount,color,value
2023-04-25,biden,large,1023591.0,bidenlarge,1023591.0
2023-04-25,biden,small,1830198.2,bidensmall,1830198.2
2023-04-26,biden,large,236328.0,bidenlarge,1259919.0
2023-04-26,biden,small,472699.85,bidensmall,2302898.05
2023-04-27,biden,large,93744.0,bidenlarge,1353663.0
2023-04-27,biden,small,215175.5,bidensmall,2518073.55
2023-04-28,biden,large,119992.0,bidenlarge,1473655.0
2023-04-28,biden,small,177722.15,bidensmall,2695795.6999999997
2023-04-29,biden,large,105940.0,bidenlarge,1579595.0
2023-04-29,biden,small,230086.6,bidensmall,2925882.3


In [28]:
data.csv(filename="daily-donor-size.csv")

CSV results at /Users/alex/projects/libfec/examples/latimes-harris-new/daily-donor-size.csv

And [the notebook](https://observablehq.com/d/0b070fb5911ecf32):



|Original|Recreation
|-|-
|![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2010.43.50%20AM.png) | ![](https://blog-static.alxg.xyz/Screen%20Shot%202024-09-05%20at%2011.49.24%20AM.png)

This one actually appears wrong! I think the cumulative sum SQL above is wrong, need to re-do this part. 